In [1]:
import pandas as pd
import pandas as pd
import numpy as np
from tqdm import notebook
import ast
import re
from fuzzywuzzy import fuzz
from datetime import datetime, timedelta
import spacy
from sklearn.model_selection import train_test_split

In [77]:
def fleiss_kappa(ratings, n):
    '''
    Computes the Fleiss' kappa measure for assessing the reliability of 
    agreement between a fixed number n of raters when assigning categorical
    ratings to a number of items.
    
    Args:
        ratings: a list of (item, category)-ratings
        n: number of raters
        k: number of categories
    Returns:
        the Fleiss' kappa score
    
    See also:
        http://en.wikipedia.org/wiki/Fleiss'_kappa
    '''
    items = set()
    categories = set()
    n_ij = {}
    
    for i, c in ratings:
        items.add(i)
        categories.add(c)
        n_ij[(i,c)] = n_ij.get((i,c), 0) + 1
    
    N = len(items)
    
    p_j = dict(((c, sum(n_ij.get((i, c), 0) for i in items) / (1.0 * n * N)) for c in categories))
    P_i = dict(((i, (sum(n_ij.get((i, c), 0) ** 2 for c in categories) - n) / (n * (n - 1.0))) for i in items))

    P_bar = sum(P_i.values()) / (1.0 * N)
    P_e_bar = sum(value ** 2 for value in p_j.values())
    
    kappa = (P_bar - P_e_bar) / (1 - P_e_bar)
    
    return kappa

In [84]:
# ratings = [(1, 'yes')] * 10 + [(1, 'no')] * 0  + \
# [(2, 'yes')] * 8  + [(2, 'no')] * 2  + \
# [(3, 'yes')] * 9  + [(3, 'no')] * 1  + \
# [(4, 'yes')] * 0  + [(4, 'no')] * 10 + \
# [(5, 'yes')] * 7  + [(5, 'no')] * 3

fleiss_kappa(ratings, 4)


0.35872058969924653

In [73]:
test_pat = pd.read_json('../../Annotation/Patrick/test_batch_patrick.json')

In [74]:
test_chiel = pd.read_json('../../Annotation/Chiel/test_batch_chiel.json')

In [83]:
ratings = []
for x in range(len(subset2_bar[0:554])):
    if subset2_bar.iloc[x].vocop_match == 0:
        ratings.append((x + 1, 'no'))
    else:
        ratings.append((x + 1, 'yes'))
    if subset2_bar.iloc[x].vocop_match == 0:
        ratings.append((x + 1, 'no'))
    else:
        ratings.append((x + 1, 'yes'))
    if test_pat.iloc[x].vocop_match == 0:
        ratings.append((x + 1, 'no'))
    else:
        ratings.append((x + 1, 'yes'))
    if test_chiel.iloc[x].vocop_match == 0:
        ratings.append((x + 1, 'no'))
    else:
        ratings.append((x + 1, 'yes'))

In [76]:
len(ratings)

2216

In [64]:
((6**2) + (548**2) - 554) / (554 * (554 - 1))

0.9785351969239005

In [65]:
pie = 0.9785351969239005 + 0.9785351969239005# + 0.971
pie = pie/2
pie

0.9785351969239005

In [68]:
pce = ((6 + 6) / (2* 554)) ** 2 + ((548 + 548) / (2* 554)) ** 2 
print(pce)
(pie - pce) / (1-pce)

0.9785739420558066


-0.0018083182640071017

In [2]:
path = 'Completed Batches/'
subset1_pat = pd.read_json(path + 'batch1_patrick_result.json')
subset2_pat = pd.read_json(path + 'batch2_patrick_result.json')
subset3_pat = pd.read_json(path + 'batch3_patrick_result.json')
subset4_pat = pd.read_json(path + 'batch4_patrick_result.json')
subset5_pat = pd.read_json(path + 'batch5_patrick_result.json')
subset1_thom = pd.read_json(path + 'batch1_thom_result.json')
subset1_chiel = pd.read_json(path + 'batch1_chiel_result.json')
subset2_chiel = pd.read_json(path + 'batch2_chiel_result.json')
subset3_chiel = pd.read_json(path + 'batch3_chiel_result.json')
subset1_bar = pd.read_json(path + 'batch1_result.json')
subset2_bar = pd.read_json(path + 'result.json')
subset3_bar = pd.read_json(path + 'batch2_result.json')
subset4_bar = pd.read_json(path + 'batch3_result.json')
subset5_bar = pd.read_json(path + 'batch4_result.json')
subset6_bar = pd.read_json(path + 'batch2_thom_result.json')

In [21]:
len(subset2_bar[subset2_bar.vocop_match == 0])

991

#### Correcties:
Chiel:
- Christiaan Andriesz 1416
- Jan de Bruijn 1463
- Abraham van de Heuvel (1823 super interessant twijfel geval)
- Pieter van Essen (1825 super interessant twijfel geval)
- Hans Pietersz 3792

Patrick:
- Jan Smit 4194 (zelde als de interessante gevallen van Chiel)
- Hendrik van Alen 593 (zelfde schip maar naam is net even anders)

In [85]:
subset1_pat.at[593, 'vocop_match'] = 0
subset1_chiel.at[1416,'vocop_match'] = 0
subset1_chiel.at[1463,'vocop_match'] = 0
subset2_chiel.at[3729, 'vocop_match'] = 0
subset4_bar.at[9025, 'vocop_match'] = ('Johannis Oosterhoff', 140442)
subset5_bar.at[9927, 'vocop_match'] = ('Wiggert Andresz', 374198)
subset6_bar.at[2670, 'vocop_match'] = ('Johan Christiaan Richter', 379155)
subset6_bar.at[2688, 'vocop_match'] = ('Jacob Meijer', 178549)
subset6_bar.at[2689, 'vocop_match'] = ('Juriaan Bartels', 178548)
subset6_bar.at[2690, 'vocop_match'] = ('Willem Borsenius', 283179)
subset6_bar.at[2691, 'vocop_match'] = ('Jan de Vaij', 182941)

In [86]:
final_pat = subset1_pat
final_pat = final_pat.append(subset2_pat)
final_pat = final_pat.append(subset3_pat).append(subset4_pat).append(subset5_pat)

final_chiel = subset1_chiel
final_chiel = final_chiel.append(subset2_chiel)
final_chiel = final_chiel.append(subset3_chiel)

final_barry = subset1_bar
final_barry = final_barry.append(subset2_bar).append(subset3_bar).append(subset4_bar).append(subset5_bar).append(subset6_bar)

final_df = final_pat.append(final_chiel).append(subset1_thom).append(final_barry)

In [87]:
final_df = final_df.loc[~final_df.index.duplicated(keep='first')]
final_df.index.name = 'index'
final_df.to_csv('../final_df.csv', index=True)
final_df[final_df.vocop_match != 0]

,uuid,rubriek,notaris,inventarisNr,akteNr,akteType,datering,taal,beschrijving,namen,urls,text,name,data_matches,data_entry,vocop_match
index,,,,,,,,,,,,,,,,
372,2dd7b308-e622-01b5-22ae-abdeba00fed2,358,JAN VERLEIJ,11889,11294,Machtiging,1744-07-02,nederlands,\nschip Nieuland; schip Zaamslag; schip Waterv...,"[{'voornaam': 'Jurriaan', 'tussenvoegsel': Non...","['KLAB05459000335.JPG', 'KLAB05459000336.JPG',...",No: 259 Grseter gepasseert den 2e: Julij 1744J...,Christiaan Morgenfeld,"[{'index': 44997, 'name_original': 'Christiaan...","{'44997': {'ships': [], 'rank': [], 'location'...","[Christiaan Morgenfeld, 298101]"
375,2dd7b308-e622-01b5-22ae-abdeba00fed2,358,JAN VERLEIJ,11889,11294,Machtiging,1744-07-02,nederlands,\nschip Nieuland; schip Zaamslag; schip Waterv...,"[{'voornaam': 'Jurriaan', 'tussenvoegsel': Non...","['KLAB05459000335.JPG', 'KLAB05459000336.JPG',...",No: 259 Grseter gepasseert den 2e: Julij 1744J...,Nanning Broeder,"[{'index': 68473, 'name_original': 'Nanning Br...","{'68473': {'ships': ['Zaamslag', 'ZAAMSLAG'], ...","[Nanning Broeder, 68473]"
376,2dd7b308-e622-01b5-22ae-abdeba00fed2,358,JAN VERLEIJ,11889,11294,Machtiging,1744-07-02,nederlands,\nschip Nieuland; schip Zaamslag; schip Waterv...,"[{'voornaam': 'Jurriaan', 'tussenvoegsel': Non...","['KLAB05459000335.JPG', 'KLAB05459000336.JPG',...",No: 259 Grseter gepasseert den 2e: Julij 1744J...,Jan Fredrik Stephaan,"[{'index': 256327, 'name_original': 'Jan Fredr...","{'256327': {'ships': [], 'rank': [], 'location...","[Jan Fredrik Stephaen, 289548]"
509,ee21c8bc-bfea-25ee-c6c1-ddf74644012b,358,JAN VERLEIJ,11890,12427,Obligatie,1744-09-12,nederlands,\nschip Diemen\n,"[{'voornaam': 'Pieter', 'tussenvoegsel': None,...","['KLAB05439000262.JPG', 'KLAB05439000263.JPG',...",Ragd werlke Getijnke Gepasseert Den 12 feptemb...,Pieter Klijn,"[{'index': 2148, 'name_original': 'Pieter Clin...","{'2148': {'ships': [], 'rank': [], 'location':...","[Pieter Klijn, 284049]"
563,35b32066-d1ee-0b29-2180-38584da36f12,358,JAN VERLEIJ,11891,11348,Obligatie,1744-10-29,nederlands,"\nschip d' Anna, VOC, 300 gulden\n","[{'voornaam': 'Nomme', 'tussenvoegsel': None, ...","['KLAB05426000116.JPG', 'KLAB05426000117.JPG',...",No: 636 Puijete Guijfiet Den 29 Eebber 1744Nte...,Cornelis Christiaansz,"[{'index': 13740, 'name_original': 'Cornelis C...","{'13740': {'ships': [], 'rank': [], 'location'...","[Cornelis Christiaansz, 329326]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2749,b8eafa41-8233-d58f-1022-092f1a8a69ee,358,JAN VERLEIJ,11915,15216,Testament,1751-11-15,nederlands,\nSchip de Immegonda\n,"[{'voornaam': 'Grietje', 'tussenvoegsel': None...","['KLAB05657000441.JPG', 'KLAB05657000442.JPG',...",Calkoen L Dit Testament van man & vrouw vervat...,Michiel Hendrik Kerken,"[{'index': 297296, 'name_original': 'Michiel H...","{'297296': {'ships': [], 'rank': [], 'location...","[Michiel Hendrik Kerke, 409527]"
2834,1420a408-ff1f-a7c4-6092-db5d4e4e5212,358,JAN VERLEIJ,11915,18096,Machtiging,1751-10-09,nederlands,"\nschipper schip de Jager, VOC kamer Hoorn\n","[{'voornaam': 'Govert', 'tussenvoegsel': None,...","['KLAB05657000178.JPG', 'KLAB05657000179.JPG',...",No: 371 Procuratie gecasseerd den 4e: October ...,Jacob Leertouwen,"[{'index': 268969, 'name_original': 'Jacob Lee...","{'268969': {'ships': [], 'rank': [], 'location...","[Jacob Leertouwer, 311020]"
2880,0f0f6da3-5628-3b6a-c33d-6104c6ade355,358,JAN VERLEIJ,11915,20001,Machtiging,1751-12-28,nederlands,\nschip Crabbendijk\n,"[{'voornaam': 'Johan', 'tussenvoegsel': 'van',...","['KLAB05657000615.JPG', 'KLAB05657000616.JPG',...",No: 505 Procuratie gecasseerd den 28e: decembr...,Johan van Veldhuijsen,"[{'index': 418944, 'name_original': 'Johan van...","{'418944': {'ships': ['Krabbendijke'], 'rank':...","[Johan van Velthuijsen, 418944]"


In [98]:
final_df.loc[5687].data_matches

[{'index': 476312,
  'name_original': 'Elias Stodius',
  'name_normalized': 'Elias Stodius',
  'date_out': '1760-03-26',
  'date_return': '1762-06-24',
  'ship_out': 'Bosschenhove',
  'ship_return': 'ZUID-BEVELAND',
  'rank': 'Bosschieter',
  'place_of_origin': 'Stokholm'},
 {'index': 489721,
  'name_original': 'Elias Stodius',
  'name_normalized': 'Elias Stodius',
  'date_out': '1754-11-02',
  'date_return': '1757-08-14',
  'ship_out': 'Herstelder',
  'ship_return': 'LUXEMBURG',
  'rank': 'Matroos',
  'place_of_origin': 'Stokkolm'},
 {'index': 493407,
  'name_original': 'Elias Stodius',
  'name_normalized': 'Elias Stodius',
  'date_out': '1762-10-31',
  'date_return': '1764-07-27',
  'ship_out': 'Westerveld',
  'ship_return': 'WESTERVELD',
  'rank': 'Kwartiermeester',
  'place_of_origin': 'Stokholm'},
 {'index': 525806,
  'name_original': 'Elias Stodlius',
  'name_normalized': 'Elias Stodlius',
  'date_out': '1757-12-30',
  'date_return': '1759-10-03',
  'ship_out': 'Renswoude',
  'sh

In [6]:
nlp = spacy.load('nl_core_news_sm', disable=['parser', 'tagger', 'textcat'])

In [7]:
def match_neighbour(start, end, true, prev, distance):
    if true == []:
        return (start, end), prev.i
    if prev.i == len(prev.doc) - 1:
        return (start, end), prev.i
    if fuzz.ratio(true[0].lower(), prev.nbor().text.lower()) >= distance:
        return match_neighbour(start, prev.nbor().idx + len(prev.nbor()), true[1:], prev.nbor(), distance)
    else:
        return (start, end), prev.i

def match_finder(row, match, distance):

    true = match
    doc = nlp(row.text)
    locs = []
    prev = 0
    for token in doc:
            for x in true:
                if token.i > prev and type(x) == str:
                    split = x.split(' ')
                    if fuzz.ratio(split[0].lower(), token.text.lower()) >= distance:
                        result, prev = match_neighbour(token.idx, token.idx + len(token), split[1:], token, distance)
                        if result not in locs:
                            if fuzz.ratio(row.text[result[0]:result[1]], x) >= distance:
                                locs.append(result)
    entities = [row.text[x[0]:x[1]] for x in locs]
    return entities

def fix_entries_ships(row, x):
    ships = match_finder(row, [x['shipOutward'], x['shipReturn']], 80)
    rank =  match_finder(row, [x['dutch_rank']], 80)
    location = match_finder(row, [x['placeOfOrigin']], 90)
#     keywords = match_finder(row, ['kamer van zeeland', 'kamer zeeland', 'kamer van amsterdam', 'kamer amsterdam', 
#                                   'kamer van hoorn', 'kamer hoorn', 'kamer enkhuizen', 'kamer van enkhuizen',
#                                   'kamer delft', 'kamer van delft', 'oostindische compagnie', 'oostindie compagnie',
#                                   'oostindie'], 80)
    true_data_matches = {'ships': ships, 'location':location, 'rank':rank}#, 'keywords':keywords}
    return true_data_matches

In [12]:
voc = pd.read_csv('../vocop_clustered_dutchrank.csv')

D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (17,27,35,36,38) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [147]:
ranking_df = pd.DataFrame(columns=['name_ratio', 'name_count', 'day_dif', 'location', 'rank', 'numships', 'keywords', 'match', 'notary_id', 'voc_id'])
name_dict = voc.fullNameNormalized.value_counts()
for x in notebook.tqdm(final_df[final_df.data_matches != 0].itertuples(), total=final_df[final_df.data_matches != 0].shape[0]):
    notary_date = datetime.strptime(x.datering, '%Y-%m-%d')
    notid = x.Index
    for match in x.data_matches:
        index = str(x.Index) + '_' + str(match['index'])
        vocid=match['index']
        name_ratio=max(fuzz.ratio(x.name, match['name_original']), fuzz.ratio(x.name, match['name_normalized']))
        name_count = name_dict[match['name_normalized']]
        
        if x.vocop_match != 0 and match['index'] == x.vocop_match[1]:
            matched = 1
        else:
            matched = 0
        
        try:
            out_date = datetime.strptime(match['date_out'], '%Y-%m-%d')
        except:
            out_date = datetime(year=1, month=1, day =1 )
        try:
            return_date = datetime.strptime(match['date_return'], '%Y-%m-%d')
        except:
            return_date = datetime(year=1, month=1, day =1 )
        if (notary_date - out_date).days in range(0, -91, -1):
            day_dif = -(notary_date - out_date).days
        elif (notary_date - return_date).days in range(0, 91):
            day_dif = (notary_date - return_date).days
        else:
            continue
        
        
        if str(match['index']) in x.data_entry:
            ship_matches = fix_entries_ships(x, match)
            numships = len(ship_matches['ships'])
            if ship_matches['rank'] != []:
                rank = 1
            else:
                rank = 0
            if ship_matches['location'] != []:
                location = 1
            else:
                location = 0
            if ship_matches['keywords'] != []:
                keywords = 1
            else:
                keywords = 0
        else:
            numships = 0
            rank = 0
            location = 0
        tempdf = pd.DataFrame(columns=['name_ratio', 'name_count', 'day_dif', 'location', 'rank', 'numships', 'keywords', 'match', 'notary_id', 'voc_id'], data={
            'name_ratio':name_ratio,
            'name_count':name_count,
            'day_dif':day_dif, 
            'location':location,
            'rank':rank,
            'numships':numships,
            'keywords':keywords,
            'match':matched,
            'notary_id':notid,
            'voc_id':vocid
        }, index=[index])
        ranking_df = ranking_df.append(tempdf)

KeyError: 'shipOutward'

In [55]:
ranking_df.to_csv('../preranking.csv', index=False)
#final_df.to_csv('../final_df.csv')
#ranking_df[ranking_df.match == 1]

In [8]:
train, test = train_test_split(final_df, test_size=0.3, random_state=10)

In [9]:
def match_neighbour(start, end, true, prev, distance):
    if true == []:
        return (start, end), prev.i
    if prev.i == len(prev.doc) - 1:
        return (start, end), prev.i
    if fuzz.ratio(true[0].lower(), prev.nbor().text.lower()) >= distance:
        return match_neighbour(start, prev.nbor().idx + len(prev.nbor()), true[1:], prev.nbor(), distance)
    else:
        return (start, end), prev.i

def match_finder(row, match, distance):

    true = match
    doc = nlp(row.text)

    locs = []
    prev = 0
    for token in doc:
        for x in true:
            if token.i > prev and type(x) == str:
                split = x.split(' ')
                if fuzz.ratio(split[0].lower(), token.text.lower()) >= distance:
                    result, prev = match_neighbour(token.idx, token.idx + len(token), split[1:], token, distance)
                    if result not in locs:
                        if fuzz.ratio(row.text[result[0]:result[1]], x) >= distance:
                            locs.append(result)
                    elif len(split) > 1 and fuzz.ratio(split[1].lower(), token.text.lower()) >= distance:
                        result, prev = match_neighbour(token.nbor(-1).idx, token.idx + len(token), split[1:], token, distance)
                        if result not in locs:
                            if fuzz.ratio(row.text[result[0]:result[1]], x) >= distance:
                                locs.append(result)
    entities = [row.text[x[0]:x[1]] for x in locs]
    return entities

def fix_entries_ships(row, x, column):
    if type(x[column]) != float: 
        match = match_finder(row, [x[column]], 80) 
    else:
        match = ''
    return match

## Dedupe Record Linking

In [13]:
data_n = {}
data_v = {}
df = train
for x in notebook.tqdm(range(len(df))):
    row = df.iloc[x]
    holder = []
    if row.data_matches != 0:
        notary_date = datetime.strptime(row.datering, '%Y-%m-%d')
        
        for match in row.data_matches:
            try:
                out_date = datetime.strptime(match['date_out'], '%Y-%m-%d')
            except:
                out_date = datetime(year=1, month=1, day =1 )
            try:
                return_date = datetime.strptime(match['date_return'], '%Y-%m-%d')
            except:
                return_date = datetime(year=1, month=1, day =1 )
            if (notary_date - out_date).days in range(0, -91, -1):
                day_dif = -(notary_date - out_date).days
            elif (notary_date - return_date).days in range(0, 91):
                day_dif = (notary_date - return_date).days
            else:
                continue
            
            found_ship_out = fix_entries_ships(row, voc.iloc[match['index']], 'shipOutward')
            found_ship_return = fix_entries_ships(row, voc.iloc[match['index']], 'shipReturn')
            found_loc = fix_entries_ships(row, voc.iloc[match['index']], 'placeOfOrigin')
            found_rank = fix_entries_ships(row, voc.iloc[match['index']], 'dutch_rank')
            holder.append({'rank':found_rank, 'location':found_loc, 'found_ship_return': found_ship_return, 'found_ship_out': found_ship_out})
            
            if row.vocop_match != 0 and match['index'] == row.vocop_match[1]:
                cluster = df.iloc[x].name
            else:
                cluster = -int(voc.iloc[match['index']].VOCOP_id)
            name = str(voc.iloc[match['index']].fullNameOriginal)
            rang = str(voc.iloc[match['index']].dutch_rank)
            loc = str(voc.iloc[match['index']].placeOfOrigin)
            ship_out = str(voc.iloc[match['index']].shipOutward) 
            ship_return = str(voc.iloc[match['index']].shipReturn)
            data_v[-int(voc.iloc[match['index']].VOCOP_id)] = {'cluster_id': cluster, 'name':name, 'rank':rang, 'location':loc, 'ship_out': ship_out, 'ship_return': ship_return}
    name = row['name']
    if holder == []:
        rang = None
        location = None
        ship_out= None
        ship_return = None
    
    else:
        rang = ' | '.join(set([y.lower() for x in holder for y in x['rank']]))
        if rang == '':
            rang = None
        location = ' | '.join(set([y.lower() for x in holder for y in x['location']]))
        if location == '':
            location = None
        ship_out = ' | '.join(set([y.lower() for x in holder for y in x['found_ship_out']]))
        if ship_out == '':
            ship_return = None
        ship_return = ' | '.join(set([y.lower() for x in holder for y in x['found_ship_return']]))
        if ship_return == '':
            ship_return = None
    data_n[df.iloc[x].name] = {'cluster_id': df.iloc[x].name, 'name':name, 'rank':rang, 'location':location, 'ship_out': ship_out, 'ship_return': ship_return}

d_n = {'cluster_id': [data_n[x]['cluster_id'] for x in data_n], 'name': [data_n[x]['name'] for x in data_n], 'rank':[data_n[x]['rank'] for x in data_n],
     'location':[data_n[x]['location'] for x in data_n], 'ship_out': [data_n[x]['ship_out'] for x in data_n], 'ship_return': [data_n[x]['ship_return'] for x in data_n]}

d_v = {'cluster_id': [data_v[x]['cluster_id'] for x in data_v], 'name': [data_v[x]['name'] for x in data_v], 'rank':[data_v[x]['rank'] for x in data_v],
     'location':[data_v[x]['location'] for x in data_v], 'ship_out': [data_v[x]['ship_out'] for x in data_v], 'ship_return': [data_v[x]['ship_return'] for x in data_v]}

dedupe_notary = pd.DataFrame(d_n)
dedupe_notary.index.name = 'index'
dedupe_notary.to_csv('../train_notary.csv')
dedupe_voc = pd.DataFrame(d_v)
dedupe_voc.index.name = 'index'
dedupe_voc.to_csv('../train_voc.csv')

In [14]:
data_n = {}
data_v = {}
df = test
for x in notebook.tqdm(range(len(df))):
    row = df.iloc[x]
    holder = []
    if row.data_matches != 0:
        notary_date = datetime.strptime(row.datering, '%Y-%m-%d')
        
        for match in row.data_matches:
            try:
                out_date = datetime.strptime(match['date_out'], '%Y-%m-%d')
            except:
                out_date = datetime(year=1, month=1, day =1 )
            try:
                return_date = datetime.strptime(match['date_return'], '%Y-%m-%d')
            except:
                return_date = datetime(year=1, month=1, day =1 )
            if (notary_date - out_date).days in range(0, -91, -1):
                day_dif = -(notary_date - out_date).days
            elif (notary_date - return_date).days in range(0, 91):
                day_dif = (notary_date - return_date).days
            else:
                continue
            
            found_ship_out = fix_entries_ships(row, voc.iloc[match['index']], 'shipOutward')
            found_ship_return = fix_entries_ships(row, voc.iloc[match['index']], 'shipReturn')
            found_loc = fix_entries_ships(row, voc.iloc[match['index']], 'placeOfOrigin')
            found_rank = fix_entries_ships(row, voc.iloc[match['index']], 'dutch_rank')
            holder.append({'rank':found_rank, 'location':found_loc, 'found_ship_return': found_ship_return, 'found_ship_out': found_ship_out})
            
            if row.vocop_match != 0 and match['index'] == row.vocop_match[1]:
                cluster = df.iloc[x].name
            else:
                cluster = -int(voc.iloc[match['index']].VOCOP_id)
            name = str(voc.iloc[match['index']].fullNameOriginal)
            rang = str(voc.iloc[match['index']].dutch_rank)
            loc = str(voc.iloc[match['index']].placeOfOrigin)
            ship_out = str(voc.iloc[match['index']].shipOutward) 
            ship_return = str(voc.iloc[match['index']].shipReturn)
            data_v[-int(voc.iloc[match['index']].VOCOP_id)] = {'cluster_id': cluster, 'name':name, 'rank':rang, 'location':loc, 'ship_out': ship_out, 'ship_return': ship_return}
    name = row['name']
    if holder == []:
        rang = None
        location = None
        ship_out= None
        ship_return = None
    
    else:
        rang = ' | '.join(set([y.lower() for x in holder for y in x['rank']]))
        if rang == '':
            rang = None
        location = ' | '.join(set([y.lower() for x in holder for y in x['location']]))
        if location == '':
            location = None
        ship_out = ' | '.join(set([y.lower() for x in holder for y in x['found_ship_out']]))
        if ship_out == '':
            ship_return = None
        ship_return = ' | '.join(set([y.lower() for x in holder for y in x['found_ship_return']]))
        if ship_return == '':
            ship_return = None
    data_n[df.iloc[x].name] = {'cluster_id': df.iloc[x].name, 'name':name, 'rank':rang, 'location':location, 'ship_out': ship_out, 'ship_return': ship_return}

d_n = {'cluster_id': [data_n[x]['cluster_id'] for x in data_n], 'name': [data_n[x]['name'] for x in data_n], 'rank':[data_n[x]['rank'] for x in data_n],
     'location':[data_n[x]['location'] for x in data_n], 'ship_out': [data_n[x]['ship_out'] for x in data_n], 'ship_return': [data_n[x]['ship_return'] for x in data_n]}

d_v = {'cluster_id': [data_v[x]['cluster_id'] for x in data_v], 'name': [data_v[x]['name'] for x in data_v], 'rank':[data_v[x]['rank'] for x in data_v],
     'location':[data_v[x]['location'] for x in data_v], 'ship_out': [data_v[x]['ship_out'] for x in data_v], 'ship_return': [data_v[x]['ship_return'] for x in data_v]}

dedupe_notary = pd.DataFrame(d_n)
dedupe_notary.index.name = 'index'
dedupe_notary.to_csv('../test_notary.csv')
dedupe_voc = pd.DataFrame(d_v)
dedupe_voc.index.name = 'index'
dedupe_voc.to_csv('../test_voc.csv')

In [16]:
len(test[test.vocop_match != 0])

26

In [23]:
16 / 26

0.6153846153846154

In [22]:
26 * 0.68

17.68

In [244]:
dedupe_notary[dedupe_notary.isna().ship_return == False]

,cluster_id,name,rank,location,ship_out,ship_return
index,,,,,,


In [245]:
dedupe_voc

,cluster_id,name,rank,location,ship_out,ship_return
index,,,,,,
0,-1085366,Jan de Vries,Bootsman,Jouver,HUIS TER DUINE,nan
1,-423179,Fredrik Schouten,Bottelier,Amsterdam,Ida,RUITER
2,-784062,Christiaan Hansz,Matroos,Scheerbeek,Leiderdorp,IMMAGONDA
3,-1118198,Christiaan Hansz,Bosschieter,Hadersleven,Fortuin,nan
4,-1310175,Christiaan Hansen,Opperstuurman,Scheerbeek,Immagonda,IMMAGONDA
...,...,...,...,...,...,...
1024,-346766,Pieter Pieterson,Bosschieter,Adersleben,Geldermalsen,nan
1025,-796268,Pieter Pietersz,Soldaat,Holsteijn,Duinenburg,nan
1026,-842628,Pieter Pietersz,Bosschieter,Bergen,Duinenburg,nan


In [193]:
def match_neighbour(start, end, true, prev, distance):
    if true == []:
        return (start, end), prev.i
    if prev.i == len(prev.doc) - 1:
        return (start, end), prev.i
    if fuzz.ratio(true[0].lower(), prev.nbor().text.lower()) >= distance:
        return match_neighbour(start, prev.nbor().idx + len(prev.nbor()), true[1:], prev.nbor(), distance)
    else:
        return (start, end), prev.i

def match_finder(row, match, distance):

    true = match
    doc = nlp(row.text)
    locs = []
    prev = 0
    for token in doc:
        for x in true:
            if type(x) == str:
                new_token = []
                split = x.split(' ')
                for y in range(len(split)):
                    if token.i <= doc[-len(split)].i:
                        new_token.append(token.nbor(y).text)
                new_token_text = ' '.join(new_token)
                if fuzz.ratio(x.lower(), new_token_text.lower()) >= distance:
                    result = (token.idx, token.idx + len(new_token_text))
                    if result not in locs:
                        if fuzz.ratio(row.text[result[0]:result[1]].lower(), x.lower()) >= distance:
                            locs.append(result)
#                 if fuzz.ratio(split[0].lower(), token.text.lower()) >= distance:
#                     result, prev = match_neighbour(token.idx, token.idx + len(token), split[1:], token, distance)
#                     if result not in locs:
#                         if fuzz.ratio(row.text[result[0]:result[1]].lower(), x.lower()) >= distance:
#                             locs.append(result)
#     if locs == []:
#         for token in doc:
#             for x in true:
#                 if type(x) == str:
#                     split = x.split(' ')
#                     joined = ''.join(split)
#                     if fuzz.ratio(x.lower(), token.text.lower()) >= distance:
#                         result, prev = match_neighbour(token.idx, token.idx + len(token), x.lower(), token, distance)
#                         if result not in locs:
#                             if fuzz.ratio(row.text[result[0]:result[1]].lower(), x.lower()) >= distance:
#                                 locs.append(result)
    entities = [row.text[x[0]:x[1]] for x in locs]
    return entities

def fix_entries_ships(row, x, column):
    if type(x[column]) != float: 
        match = match_finder(row, [x[column]], 80) 
    else:
        match = ''
    return match

In [182]:
final_df[final_df.vocop_match != 0].loc[2689].text

"No: 295 Verklaring gecasseerd den 19e: apr: 1755GeennnOp heden den 19e: Augustes des Jaars 1755 Compareerde voor mij Jan verleij Notaris publiek bij den Ed: Hove van Holland ge admitteerd binnen Amsterdam resideerende d' E. E: Jurriaani295 den 19 Augt d' E. E. Jurriaan Bartels voor Luitenant en Willem Borzenuis voor onderstuurman nu 1754 Laast met het Schip 't huijs ten Duijnen uijt g Oostindien thuijs gekomen; En hebben Sij Compa� ranten die van genoegzamen ouderdom Sijn ter ^ mede requisitie van Jacob Meijer ^ voor onderstuurman met het gemelde Schip gerepatrieert; en ten behoeve van die 't verder mogte aangaan, getuijgt & verklaerd voornoemde Dat Sij betuigen op de thuijsreijse vang Schip van Bengale herwaerts, den opperstuurman Jan de Vrij (die nevens den requirant en den opper� meester harmanus Kluwen gemagtigdes was van den overledene Krank besoeker gerbrand ^ in duijdelijke bewoordingen Hendriksz) verscheijde malen ^ hebben horen Seggen dat de nagelate goederen van gemelde Kran

In [183]:
fuzz.ratio('duine', 'duijnen')

83

In [184]:
doc = nlp('stinky doodoo in je stinky poopoo')
for token in doc:
    new_token = ['']
    for y in range(len('Huis te Duine'.split(' '))):
        if token.i <= doc[len('Huis te Duine'.split(' '))].i:
            new_token.append(token.nbor(y).text)
    print(' '.join(new_token))
        

 stinky doodoo in
 doodoo in je
 in je stinky
 je stinky poopoo




In [194]:
c = 0
total = 0
for x in notebook.tqdm(final_df[final_df.vocop_match != 0].itertuples(), total=final_df[final_df.vocop_match != 0].shape[0]):
    for y in x.data_matches:
        if y['index'] == x.vocop_match[1]:
            test = fix_entries_ships(x, voc.iloc[y['index']], 'shipOutward')
            if test != '':
                total += 1
            if test not in [[], '']:
                c += 1
#             else:
#                 print(x.Index)
#                 print(x.beschrijving)
#                 print(voc.iloc[y['index']]['shipOutward'])
                
                
print(c, total)


70 101


In [195]:
c = 0
total = 0
for x in notebook.tqdm(final_df[final_df.vocop_match != 0].itertuples(), total=final_df[final_df.vocop_match != 0].shape[0]):
    for y in x.data_matches:
        if y['index'] == x.vocop_match[1]:
            test = fix_entries_ships(x, voc.iloc[y['index']], 'shipReturn')
            if test != '':
                total += 1
            if test not in [[], '']:
                c += 1
                
print(c, total)


51 86


In [191]:
c = 0
total = 0
for x in notebook.tqdm(final_df[final_df.vocop_match != 0].itertuples(), total=final_df[final_df.vocop_match != 0].shape[0]):
    for y in x.data_matches:
        if y['index'] == x.vocop_match[1]:
            test = fix_entries_ships(x, voc.iloc[y['index']], 'dutch_rank')
            if test != '':
                total += 1
            if test not in [[], '']:
                c += 1
                
print(c, total)


38 101


In [192]:
c = 0
total = 0
for x in notebook.tqdm(final_df[final_df.vocop_match != 0].itertuples(), total=final_df[final_df.vocop_match != 0].shape[0]):
    for y in x.data_matches:
        if y['index'] == x.vocop_match[1]:
            test = fix_entries_ships(x, voc.iloc[y['index']], 'placeOfOrigin')
            if test != '':
                total += 1
            if test not in [[], '']:
                c += 1
                
print(c, total)


56 101
